In [ ]:
!wget --no-check-certificate https://www.dropbox.com/scl/fi/macggi1t14ui1cnooqxdv/NLPmaterial.zip?rlkey=pp713v7s9r6b0j8u718aepuby&dl=0

--2024-03-13 02:47:43--  https://www.dropbox.com/scl/fi/macggi1t14ui1cnooqxdv/NLPmaterial.zip?rlkey=pp713v7s9r6b0j8u718aepuby
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf4b1b8df28b1314167bd3b5197.dl.dropboxusercontent.com/cd/0/inline/CO8Sucgxf9IzI0Q8RVE1J-TB6wv0WSh-cGXiORvbGDdXSnoprvjqYU_U7lTz9DS9kZLYjWT3R1OEdtw37VU2-4AAoxzueeKTap-Pt4NwRQh0zopuVsbxMpN78sBlXC6natv4qWgYVZVNJopjctYc-keo/file# [following]
--2024-03-13 02:47:44--  https://ucf4b1b8df28b1314167bd3b5197.dl.dropboxusercontent.com/cd/0/inline/CO8Sucgxf9IzI0Q8RVE1J-TB6wv0WSh-cGXiORvbGDdXSnoprvjqYU_U7lTz9DS9kZLYjWT3R1OEdtw37VU2-4AAoxzueeKTap-Pt4NwRQh0zopuVsbxMpN78sBlXC6natv4qWgYVZVNJopjctYc-keo/file
Resolving ucf4b1b8df28b1314167bd3b5197.dl.dropboxusercontent.com (ucf4b1b8df28b1314167bd3b5197.dl.dropboxusercontent.com)... 162.125.4.15, 262

In [ ]:
!unzip /content/NLPmaterial.zip?rlkey=pp713v7s9r6b0j8u718aepuby

Archive:  /content/NLPmaterial.zip?rlkey=pp713v7s9r6b0j8u718aepuby
  inflating: material/input_token_index.json  
  inflating: material/s2s_gru_model100.keras  
  inflating: material/s2s_lstm_model100.keras  
  inflating: material/s2s_lstm_model20.keras  
  inflating: material/target_token_index.json  


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl siz

In [ ]:
import keras
import json
import numpy as np
import gradio as gr

In [ ]:
modelLSTM100 = keras.models.load_model("/content/material/s2s_lstm_model100.keras") #50000 simples
modelLSTM20 = keras.models.load_model("/content/material/s2s_lstm_model20.keras") #100000 simples
modelGRU = keras.models.load_model("/content/material/s2s_gru_model100.keras") #50000 simples

In [ ]:
num_decoder_tokens = 36
num_encoder_tokens = 85
latent_dim = 256
max_decoder_seq_length = 43
max_encoder_seq_length = 29

In [ ]:
with open('/content/material/target_token_index.json', 'r') as json_file:
    target_token_index = json.load(json_file)

with open('/content/material/input_token_index.json', 'r') as json_file:
    input_token_index = json.load(json_file)

In [ ]:
encoder_inputs_lstm = modelLSTM100.input[0]  # input_1
encoder_outputs__lstm, state_h_enc_lstm, state_c_enc_lstm = modelLSTM100.layers[2].output  # lstm_1
encoder_states__lstm = [state_h_enc_lstm, state_c_enc_lstm]
encoder_model__lstm = keras.Model(encoder_inputs_lstm, encoder_states__lstm)

decoder_inputs_lstm = modelLSTM100.input[1]  # input_2
decoder_state_input_h_lstm = keras.Input(shape=(latent_dim,))
decoder_state_input_c__lstm = keras.Input(shape=(latent_dim,))
decoder_states_inputs_lstm = [decoder_state_input_h_lstm, decoder_state_input_c__lstm]
decoder_lstm = modelLSTM100.layers[3]
decoder_outputs_lstm, state_h_dec_lstm, state_c_dec_lstm = decoder_lstm(
    decoder_inputs_lstm, initial_state=decoder_states_inputs_lstm
)
decoder_states_lstm = [state_h_dec_lstm, state_c_dec_lstm]
decoder_dense_lstm = modelLSTM100.layers[4]
decoder_outputs_lstm = decoder_dense_lstm(decoder_outputs_lstm)
decoder_model_lstm = keras.Model(
    [decoder_inputs_lstm] + decoder_states_inputs_lstm, [decoder_outputs_lstm] + decoder_states_lstm
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence_lstm(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model__lstm.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    # print("tar "+ target_seq)
    # print("sta "+ states_value)
    #print(f"tar: {target_seq.shape}")
    #print(f"sta: {states_value}")
    #print(type(states_value))
    while not stop_condition:
        output_tokens, h, c = decoder_model_lstm.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
encoder_inputs_lstm2 = modelLSTM20.input[0]  # input_1
encoder_outputs__lstm2, state_h_enc_lstm2, state_c_enc_lstm2 = modelLSTM20.layers[2].output  # lstm_2
encoder_states__lstm2 = [state_h_enc_lstm2, state_c_enc_lstm2]
encoder_model__lstm2 = keras.Model(encoder_inputs_lstm2, encoder_states__lstm2)

decoder_inputs_lstm2 = modelLSTM20.input[1]  # input_2
decoder_state_input_h_lstm2 = keras.Input(shape=(latent_dim,))
decoder_state_input_c__lstm2 = keras.Input(shape=(latent_dim,))
decoder_states_inputs_lstm2 = [decoder_state_input_h_lstm2, decoder_state_input_c__lstm2]
decoder_lstm2 = modelLSTM20.layers[3]
decoder_outputs_lstm2, state_h_dec_lstm2, state_c_dec_lstm2 = decoder_lstm2(
    decoder_inputs_lstm2, initial_state=decoder_states_inputs_lstm2
)
decoder_states_lstm2 = [state_h_dec_lstm2, state_c_dec_lstm2]
decoder_dense_lstm2 = modelLSTM20.layers[4]
decoder_outputs_lstm2 = decoder_dense_lstm2(decoder_outputs_lstm2)
decoder_model_lstm2 = keras.Model(
    [decoder_inputs_lstm2] + decoder_states_inputs_lstm2, [decoder_outputs_lstm2] + decoder_states_lstm2
)

# Reverse-lookup token index to decode sequences back to
# something readable.
# reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence_lstm2(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model__lstm2.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    # print("tar "+ target_seq)
    # print("sta "+ states_value)
    #print(f"tar: {target_seq.shape}")
    #print(f"sta: {states_value}")
    #print(type(states_value))
    while not stop_condition:
        output_tokens, h, c = decoder_model_lstm2.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
def encode_input(name):
    test_input = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    for t, char in enumerate(name):
        test_input[0, t, input_token_index[char]] = 1.
    return test_input

In [ ]:
encoder_inputs_gru = modelGRU.input[0]  # input_1
encoder_outputs_gru, state_h_enc_gru = modelGRU.layers[2].output  # gru_1
encoder_states_gru = [state_h_enc_gru]
encoder_model_gru = keras.Model(encoder_inputs_gru, encoder_states_gru)

decoder_inputs_gru = modelGRU.input[1]  # input_2
decoder_state_input_h_gru = keras.Input(shape=(latent_dim,))
#decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs_gru = [decoder_state_input_h_gru]
decoder_gru = modelGRU.layers[3]
decoder_outputs_gru, state_h_dec_gru = decoder_gru(
    decoder_inputs_gru, initial_state=decoder_states_inputs_gru
)
decoder_states_gru = [state_h_dec_gru]
decoder_dense_gru = modelGRU.layers[4]
decoder_outputs_gru = decoder_dense_gru(decoder_outputs_gru)
decoder_model_gru = keras.Model(
    [decoder_inputs_gru] + decoder_states_inputs_gru, [decoder_outputs_gru] + decoder_states_gru
)

# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
#reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence_gru(input_seq):
    states_value = encoder_model_gru.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model_gru.predict([target_seq] + [np.array(states_value)])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = h
    return decoded_sentence

In [ ]:
def decode_sequence_gru(input_seq):
    states_value = encoder_model_gru.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model_gru.predict([target_seq] + [np.array(states_value)])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = h
    return decoded_sentence

In [ ]:
def encode_input(name):
    test_input = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    for t, char in enumerate(name):
        test_input[0, t, input_token_index[char]] = 1.
    return test_input

def rom_lstm100(text):
	return decode_sequence_lstm(encode_input(text))

def rom_lstm20(text):
	return decode_sequence_lstm2(encode_input(text))

def rom_gru(text):
	return decode_sequence_gru(encode_input(text))

In [ ]:
r1 = rom_lstm100('สวัสดีไก่ปลา')
r2 = rom_lstm20('สวัสดีไก่ปลา')
r3 = rom_gru('สวัสดีไก่ปลา')

1/1 [==============================] - 0s 23ms/step


In [ ]:
print(f"lstm100: {r1}")
print(f"lstm20: {r2}")
print(f"gru: {r3}")

lstm100: sawatdiakkilap

lstm20: sawattikaipla

gru: sawatsikailak



In [ ]:
import gradio as gr

def greet(input):
    return f"LSTM100:{rom_lstm100(input)}\nLSTM20: {rom_lstm20(input)}\nGRU: {rom_gru(input)}"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            thai = gr.Textbox(label="Thai Text")
            translate_btn = gr.Button(value="Translate")
        with gr.Column():
            roman = gr.Textbox(label="Romanrize Text")
    translate_btn.click(greet, inputs=thai, outputs=roman)
    examples = gr.Examples(examples=["แพลนนิง","นิสัย","ลาก่อน","พรุ่งนี้"],inputs=thai)
# demo = gr.Interface(fn=greet, inputs=gr.Textbox(lines= 2, placeholder='Text here'), outputs="textbox")

demo.launch(share=True)  # Share your demo with just 1 extra parameter 🚀

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://21140fd437d1b9186a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
